In [8]:
import pandas as pd
import joblib
import gradio as gr
from huggingface_hub import hf_hub_download

In [9]:
REPO = "YShutko/spotify-popularity-models"

MODEL_FILE = "xgb_model_best.pkl"     # or "xgb_model_best.pkl" or "linear_regression_model.pkl"

model_path = hf_hub_download(
    repo_id=REPO,
    filename=MODEL_FILE,
    token=None  # required only if repo or file is private
)

model = joblib.load(model_path)

print("Model loaded successfully:", MODEL_FILE)


Model loaded successfully: xgb_model_best.pkl


In [10]:
df = pd.read_csv("../data/spotify_cleaned_data.csv")


In [11]:
# Macro-genre from track_genre
def map_macro_genre(g):
    g = str(g).lower()
    if "pop" in g:
        return "Pop"
    elif "rock" in g:
        return "Rock"
    elif "hip hop" in g or "rap" in g or "trap" in g:
        return "Hip-Hop/Rap"
    elif "r&b" in g or "soul" in g:
        return "R&B/Soul"
    elif "electro" in g or "techno" in g or "house" in g or "edm" in g or "dance" in g:
        return "Electronic/Dance"
    elif "metal" in g or "hardcore" in g:
        return "Metal/Hardcore"
    elif "jazz" in g or "blues" in g:
        return "Jazz/Blues"
    elif "classical" in g or "orchestra" in g or "piano" in g:
        return "Classical"
    elif "latin" in g or "reggaeton" in g or "sertanejo" in g or "samba" in g:
        return "Latin"
    elif "country" in g:
        return "Country"
    elif "folk" in g or "singer-songwriter" in g:
        return "Folk"
    elif "indie" in g or "alternative" in g:
        return "Indie/Alternative"
    else:
        return "Other"

df["macro_genre"] = df["track_genre"].apply(map_macro_genre)

In [ ]:
def predict_popularity_api(energy, danceability, valence, loudness, tempo, explicit, genre):

    sample = pd.DataFrame([{
        "energy": energy,
        "danceability": danceability,
        "valence": valence,
        "loudness": loudness,
        "tempo": tempo,
        "explicit": explicit,
        "macro_genre": genre,
        "artist_popularity": 50,
        "loudness_danceability": loudness * danceability,
        "energy_valence": energy * valence,
        "instrumentalness": 0,
        "acousticness": 0,
        "liveness": 0,
        "speechiness": 0,
        "duration_min": 3
    }])

    prediction = model.predict(sample)[0]
    return round(float(prediction), 2)


In [13]:
interface = gr.Interface(
    fn=predict_popularity_api,
    inputs=[
        gr.Slider(0, 1, step=0.01, label="Energy"),
        gr.Slider(0, 1, step=0.01, label="Danceability"),
        gr.Slider(0, 1, step=0.01, label="Valence"),
        gr.Slider(-60, 0, step=1, label="Loudness"),
        gr.Slider(50, 200, step=1, label="Tempo"),
        gr.Dropdown([True, False], label="Explicit"),
        gr.Dropdown(sorted(df["macro_genre"].unique()), label="Macro Genre")
    ],
    outputs="number",
    title="🎧 Spotify Popularity Predictor (Model from HuggingFace)"
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
